Initializing Table

In [1]:
import tabula
import pandas as pd

Functions

In [2]:
#Extracts key words on the pdf
#Returns a data frame of keywords and it's location on the pdf, with page numbers
#Step 1 in table parsing

from pathlib import Path
from typing import Iterable, Any

from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer


def extract_to_df(pages: Any):
    cols = ["Page","Text", "x_1", "y_1", "x_2", "y_2"]
    words = ["Last Raced", "Fractional Times", "Past Performance Running Line Preview", "Fin", "Trainers"]
    text_loc_df = pd.DataFrame(columns = cols)
    
    page_counter = 0
    for page_layout in pages:
        page_counter += 1
        word_counter = 0
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                element_text = element.get_text()
                for word in words:
                    if(element_text.__contains__(word)):
                        loc_nums = []
                        for num in element.bbox:
                            loc_nums.append(num) #Adding binding box numbers
                        text_loc_df.loc[len(text_loc_df.index)] = [page_counter,
                                                                     word,
                                                                     loc_nums[0], 
                                                                     loc_nums[1], 
                                                                     loc_nums[2], 
                                                                     loc_nums[3]]
    return text_loc_df

In [3]:
#Getting pixel location of words per page
#Step 2 in table parsing

def create_text_loc_df(page_num,df):
    text_loc_df = df.loc[df['Page'] == page_num] 

    #Saving only last instance of fin; Lowest y_1

    fin_df = text_loc_df.loc[text_loc_df['Text'] == "Fin"] #df with only Fin
    fin_df = fin_df.sort_values(by=['y_1']) #Sort by y_1 descending
    fin_row = fin_df.iloc[0] #Save 1st row
    text_loc_df = text_loc_df.loc[text_loc_df['Text'] != "Fin"] #Delete Fin rows
    #text_loc_df = pd.concat(text_loc_df,fin_row.to_frame(),ignore_index=True) #Append right Fin row
    text_loc_df.loc[len(text_loc_df) + 1] = fin_row

    #Resort table by index
    text_loc_df = text_loc_df.sort_index()
    return text_loc_df

In [4]:
#Creating table location dataframe
#Converts to inches and then pdf_location for tabula
#Step 3 of table parsing

def table_location_to_df(page_num,text_loc_df):
    df = text_loc_df
    cols = ["Page","Table",'Top','Left','Bottom','Right']
    table_loc_df = pd.DataFrame(columns = cols)
    page = page_num
    #Top Table

    left = 7.92
    right = 559.44

    y_1 = df.loc[df['Text'] == "Last Raced",'y_1'].tolist()[0] #Getting y_1 value
    top = (((729 - y_1) * 11)/792) *72

    y_1 = df.loc[df['Text'] == "Fractional Times",'y_1'].tolist()[0]
    bottom = (((729 - y_1) * 11)/792 + .50) *72

    #Adding Table 1 to df
    table_loc_df.loc[len(table_loc_df.index)] =[page,1,top,left,bottom,right]

    #Bottom Table

    left = 126
    right = 424.08

    y_1 = df.loc[df['Text'].str.contains("Past Performance"),'y_1'].tolist()[0]
    top = (((729 - y_1) * 11)/792) * 72


    y_1 = df.loc[df['Text'] == "Trainers",'y_1'].tolist()[0]
    bottom = ((((729 - y_1) * 11)/792 + .75)) * 72

    #Adding Table 2 to df
    table_loc_df.loc[len(table_loc_df.index)] =[page,2,top,left,bottom,right]
    return table_loc_df


In [5]:
#Gets the non-empty pages and number of horses per page

from PyPDF2 import PdfReader
def get_page_list(file):  #Pdf file
    reader = PdfReader(file)
    num_pages = len(reader.pages)
    
    
    page_list = []
    for page_num in range(num_pages + 1):
        page = reader.pages[page_num - 1]
        text = page.extract_text()
        tokenized_text = text.split()

        start_count_bool = False
        count = 0
        for i in range(0,len(tokenized_text) - 1):
            if(tokenized_text[i] == "Trainers:"):
                start_count_bool = True
            elif(tokenized_text[i] == "Owners:"):
                start_count_bool = False
            elif(start_count_bool and tokenized_text[i] == '-'):
                count += 1

        if count > 0:
            page_list.append({'page_num' : page_num, 'horse_count' : count})
    return page_list

In [6]:
#Gets a table from a pdf, using a dataframe of rough locations of the tables on the pdf
#Returns a dataframe of the table
#Step 4 of table parsing


def get_table(file,
              table_loc_df, #Dataframe of locations of tables on pdf
              table_num, #Table we are parsing for (1 = top, 2 = bottom)
              page, #Page number
              num_horses): #Number of horses on page
    print("Looking for tables...")
    #Setting variables 
    num_target_rows = num_horses * 2
    
    #Initial table scan
    try:
        scan_df = tabula.read_pdf(file, pages = page, area = [test_area])
        table_df = scan_df[0]
        print(table_df)
        col_names = list(table_df.columns.values)
        num_rows = len(table_df.index)
    except:
        print("Couldn't read a table from default area")
        col_names = ["top_not_found"]
        num_rows = 0
        
    #Finding top bound
    
    
    #Setting header value to look for
    if(table_num == 1):
        target_header = 'Last Raced'
    else:
        target_header = 'Pgm'
        

    
    #Loop until top bound is right
    bound_num = 0 #Increased bounds by 10, both positive and negative to find the right headers. Ex: 10, -10, 20, -20, 30...
    while(col_names[0] != target_header or bound_num > 200):        
        top_bound = table_loc_df.iloc[table_num - 1,2]
        top_bound += bound_num
        test_area = [
            top_bound,
            table_loc_df.iloc[table_num - 1,3],
            table_loc_df.iloc[table_num - 1,4],
            table_loc_df.iloc[table_num - 1,5]
        ]
        try:
            scan_df = tabula.read_pdf(file, pages = page, area = [test_area])
            table_df = scan_df[0]
        except:
            print("Couldn't read a table, moving on from {} top_bound".format(top_bound))
            
        col_names = list(table_df.columns.values)

        #Change bound_num
        if(bound_num <= 0): #If number is in negative cycle
            bound_num *= -1
            bound_num += 10
        else:
            bound_num *= -1
    
    if(bound_num > 200):
        print("Error: Couldn't find top bound")
        return
    else:
        print("Found table top bound at {}".format(top_bound))
        table_loc_df.iloc[table_num - 1, 2] = top_bound #Adding new top bound to df
    
    #Finding bottom bound

    bound_num = 0 #Increased bounds by 10, both positive and negative to find the right headers. Ex: 10, -10, 20, -20, 30...
    while(num_rows != num_target_rows or bound_num > 200):
        bottom_bound = table_loc_df.iloc[table_num - 1,2]
        bottom_bound += bound_num
        test_area = [
            table_loc_df.iloc[table_num - 1,2],
            table_loc_df.iloc[table_num - 1,3],
            bottom_bound,
            table_loc_df.iloc[table_num - 1,5]
        ]
        try:
            scan_df = tabula.read_pdf(file, pages = page, area = [test_area])
            table_df = scan_df[0]
        except:
            print("Couldn't read a table, moving on from {} bottom_bound".format(bottom_bound))
            
        num_rows = len(table_df.index)

         #Change bound_num
        if(bound_num <= 0): #If number is in negative cycle
            bound_num *= -1
            bound_num += 10
        else:
            bound_num *= -1
    
    if(bound_num > 200):
        print("Error: Couldn't find bottom bound")
    else:
        table_loc_df.iloc[table_num - 1, 4] = bottom_bound #Adding new bottom bound to df
        print("Found table bottom bound at {}".format(bottom_bound))
        return table_df

In [7]:
import warnings
warnings.filterwarnings('ignore')

pdf = "equibaseFile.pdf"
df_list = []
page_list = get_page_list(pdf)
print(page_list)
#Extract words on position
pages = extract_pages(pdf)
full_text_loc_df = extract_to_df(pages)
for page in page_list: #Get both tables for each page
    print("Page num: " ,page["page_num"])
    try:
        text_loc_df = create_text_loc_df(page['page_num'],full_text_loc_df) #Locations of key text on page
        table_loc_df = table_location_to_df(page['page_num'],text_loc_df) #Locations of tables on page
        top_table = get_table(pdf,table_loc_df,1,page['page_num'],page['horse_count']) #Getting top table
        bottom_table = get_table(pdf,table_loc_df,2,page['page_num'],page['horse_count']) #Getting bottom table
        #Add to list of dfs
        df_list.append(top_table)
        df_list.append(bottom_table)
    except:
        print("Error with page", page["page_num"])
        



The PDF <_io.BufferedReader name='equibaseFile.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


[{'page_num': 0, 'horse_count': 5}, {'page_num': 1, 'horse_count': 5}, {'page_num': 2, 'horse_count': 7}, {'page_num': 3, 'horse_count': 5}, {'page_num': 4, 'horse_count': 6}, {'page_num': 5, 'horse_count': 8}, {'page_num': 6, 'horse_count': 7}, {'page_num': 7, 'horse_count': 5}, {'page_num': 8, 'horse_count': 8}, {'page_num': 9, 'horse_count': 8}, {'page_num': 10, 'horse_count': 8}, {'page_num': 11, 'horse_count': 5}]
Page num:  0
Error with page 0
Page num:  1
Looking for tables...
Couldn't read a table from default area
Found table top bound at 135.24599999999995


Error from tabula-java:
Exception in thread "main" java.lang.UnsupportedOperationException: Can't add an oblique ruling.
	at technology.tabula.Page.addRuling(Page.java:302)
	at technology.tabula.Page.addBorderRulingsTo(Page.java:159)
	at technology.tabula.Page.getArea(Page.java:134)
	at technology.tabula.CommandLineApp.extractFile(CommandLineApp.java:177)
	at technology.tabula.CommandLineApp.extractFileTables(CommandLineApp.java:124)
	at technology.tabula.CommandLineApp.extractTables(CommandLineApp.java:106)
	at technology.tabula.CommandLineApp.main(CommandLineApp.java:76)




Couldn't read a table, moving on from 135.24599999999995 bottom_bound
Found table bottom bound at 135.24599999999995
Looking for tables...
Couldn't read a table from default area
Found table top bound at 404.8310000000001


Error from tabula-java:
Exception in thread "main" java.lang.UnsupportedOperationException: Can't add an oblique ruling.
	at technology.tabula.Page.addRuling(Page.java:302)
	at technology.tabula.Page.addBorderRulingsTo(Page.java:159)
	at technology.tabula.Page.getArea(Page.java:134)
	at technology.tabula.CommandLineApp.extractFile(CommandLineApp.java:177)
	at technology.tabula.CommandLineApp.extractFileTables(CommandLineApp.java:124)
	at technology.tabula.CommandLineApp.extractTables(CommandLineApp.java:106)
	at technology.tabula.CommandLineApp.main(CommandLineApp.java:76)




...

Couldn't read a table, moving on from 439.997 bottom_bound
Found table bottom bound at 439.997
Page num:  11
Looking for tables...
Couldn't read a table from default area
Found table top bound at 135.24599999999995


Error from tabula-java:
Exception in thread "main" java.lang.UnsupportedOperationException: Can't add an oblique ruling.
	at technology.tabula.Page.addRuling(Page.java:302)
	at technology.tabula.Page.addBorderRulingsTo(Page.java:159)
	at technology.tabula.Page.getArea(Page.java:134)
	at technology.tabula.CommandLineApp.extractFile(CommandLineApp.java:177)
	at technology.tabula.CommandLineApp.extractFileTables(CommandLineApp.java:124)
	at technology.tabula.CommandLineApp.extractTables(CommandLineApp.java:106)
	at technology.tabula.CommandLineApp.main(CommandLineApp.java:76)




Couldn't read a table, moving on from 135.24599999999995 bottom_bound
Found table bottom bound at 135.24599999999995
Looking for tables...
Couldn't read a table from default area
Found table top bound at 445.3310000000001


Error from tabula-java:
Exception in thread "main" java.lang.UnsupportedOperationException: Can't add an oblique ruling.
	at technology.tabula.Page.addRuling(Page.java:302)
	at technology.tabula.Page.addBorderRulingsTo(Page.java:159)
	at technology.tabula.Page.getArea(Page.java:134)
	at technology.tabula.CommandLineApp.extractFile(CommandLineApp.java:177)
	at technology.tabula.CommandLineApp.extractFileTables(CommandLineApp.java:124)
	at technology.tabula.CommandLineApp.extractTables(CommandLineApp.java:106)
	at technology.tabula.CommandLineApp.main(CommandLineApp.java:76)




Couldn't read a table, moving on from 445.3310000000001 bottom_bound
Found table bottom bound at 445.3310000000001


Cleaning Tables

In [29]:
#Top Table

def clean_top_table(df): #df of top table to be cleaned
    #Merging super scripts
    script_df = df.loc[df['Pgm'].isnull()]
    
    #Get col names of race partitions
    start_index = script_df.columns.get_loc('Start')
    end_index = script_df.columns.get_loc('Odds')
    
    col_name_list = script_df.columns[start_index + 1:end_index]
    col_name_list = [*['Last Raced'],*col_name_list]
     
    script_df = script_df.loc[:,col_name_list]
    
    col_name_list[0] = col_name_list[0] + '_super_script'
    for i in range (1,len(col_name_list)):
        col_name_list[i] = col_name_list[i] + '_length_behind'
    script_df.columns = col_name_list
    
    df_horses = df.loc[df['Pgm'].isnull()==False].reset_index(drop=False)
    script_df.reset_index(drop = True, inplace = True)
    print(script_df)
    final_df = df_horses.merge(script_df,left_index=True,right_index=True,how='left')

    #Splitting Wgt and M/E
    final_df[['Wgt', 'M/E']] = final_df['Wgt M/E'].str.split(' ', 1,expand = True)

    #Removing unnamed & old index
    final_df.drop(['index', 'Wgt M/E'], axis = 1, inplace = True)

    #Removing unnamed
    end_index = final_df.columns.get_loc('Pgm')
    for i in range(1,end_index):
        final_df.drop(final_df.columns[1],axis = 1, inplace = True)
        
    return final_df


In [30]:
df = clean_top_table(df_list[4])
df

  Last Raced_super_script 3/16_length_behind  3/8_length_behind  \
0                     NaN              2 1/2                5.0   
1                     NaN                  6                1.0   
2                     1 5                  4                7.0   
3                     NaN                NaN                1.0   
4                     NaN               Head                NaN   

  Str_length_behind Fin_length_behind  
0             3 1/2                 1  
1                 6            15 1/4  
2                 5              Neck  
3                 3             5 1/4  
4               NaN               NaN  


,Last Raced,Pgm,Horse Name (Jockey),PP,Start,3/16,3/8,Str,Fin,Odds,Comments,Last Raced_super_script,3/16_length_behind,3/8_length_behind,Str_length_behind,Fin_length_behind,Wgt,M/E
0,---,2.0,"Recruiter (Marquez, Charlie)",2.0,1.0,1,1.0,1,1,1.00*,"sharp early,asked lat",NaN,2 1/2,5.0,3 1/2,1,117,L bf
1,---,7.0,"Global Surprise (Olivero, Carlos)",4.0,3.0,3,2.0,2,2,1.10,"rallied, checked wire",NaN,6,1.0,6,15 1/4,117,L
2,12Aug22 MTH,8.0,"Cruising Derek (Moya, Ramon)",5.0,2.0,2,3.0,3,3,10.70,"chased, weakened",1 5,4,7.0,5,Neck,117,L bf
3,---,6.0,"Capt Complicated (Torres, Jomar)",3.0,5.0,5,4.0,4,4,10.50,"2w, no factor",NaN,NaN,1.0,3,5 1/4,118,L f
4,---,1.0,"Give God Praise (Mitchell, Richard)",1.0,4.0,4,5.0,5,5,11.80,"inside, outrun",NaN,Head,NaN,NaN,NaN,117,L


In [17]:
def clean_bottom_table(df): #df of top table to be cleaned
    #Merging super scripts
    script_df = df.loc[df['Pgm'].isnull()]
  
    #Get col names of race partitions
    start_index = script_df.columns.get_loc('Start')
    end_index = script_df.columns.get_loc('Fin') + 1
    
    col_name_list = script_df.columns[start_index + 1:end_index]
    
    col_name_list = [*col_name_list]
    script_df = script_df.loc[:,col_name_list]
    

    for i in range (0,len(col_name_list)):
        col_name_list[i] = col_name_list[i] + '_length_behind'
    script_df.columns = col_name_list

    df_horses = df.loc[df['Pgm'].isnull()==False].reset_index(drop=False)
    final_df = df_horses.merge(script_df,left_index=True,right_index=True,how='left')

    df_horses = df.loc[df['Pgm'].isnull()==False].reset_index(drop=False)
    final_df = df_horses.merge(script_df,left_index=True,right_index=True,how='left')


    #Removing old index
    final_df.drop('index', axis = 1, inplace = True)

    return final_df

In [26]:
df_list[4]

,Last Raced,Unnamed: 0,Pgm,Horse Name (Jockey),Wgt M/E,PP,Start,3/16,3/8,Str,Fin,Odds,Comments
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 1/2,5.0,3 1/2,1,NaN,NaN
1,---,NaN,2.0,"Recruiter (Marquez, Charlie)",117 L bf,2.0,1.0,1,1.0,1,1,1.00*,"sharp early,asked lat"
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,1.0,6,15 1/4,NaN,NaN
3,---,NaN,7.0,"Global Surprise (Olivero, Carlos)",117 L,4.0,3.0,3,2.0,2,2,1.10,"rallied, checked wire"
4,1 5,NaN,NaN,NaN,NaN,NaN,NaN,4,7.0,5,Neck,NaN,NaN
5,12Aug22 MTH,NaN,8.0,"Cruising Derek (Moya, Ramon)",117 L bf,5.0,2.0,2,3.0,3,3,10.70,"chased, weakened"
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3,5 1/4,NaN,NaN
7,---,NaN,6.0,"Capt Complicated (Torres, Jomar)",118 L f,3.0,5.0,5,4.0,4,4,10.50,"2w, no factor"
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head,NaN,NaN,NaN,NaN,NaN
9,---,NaN,1.0,"Give God Praise (Mitchell, Richard)",117 L,1.0,4.0,4,5.0,5,5,11.80,"inside, outrun"


In [19]:
#Clean all dataframes in first pdf

cleaned_df_list = []
for i in range (len(df_list)):
    if(i % 2 == 0): #If even
        try:
            cleaned_df_list.append(clean_top_table(df_list[i]))
        except:
            print("Error with df ", i)
            cleaned_df_list.append(df_list[i])
    else:
        try:
            cleaned_df_list.append(clean_bottom_table(df_list[i]))
        except:
            print("Error with df", i)
            cleaned_df_list.append(df_list[i])


   index   Last Raced  Unnamed: 0  Pgm  \
0      1   2Jul22 MTH         NaN  4.0   
1      3   2Jul22 MTH         NaN  6.0   
2      5  30Jun22 DEL         NaN  1.0   
3      7  22Jul22 MTH         NaN  2.0   
4      9  30Jul22 MTH         NaN  3.0   

                       Horse Name (Jockey)  Wgt M/E   PP  Start 1/4 3/8 Str  \
0               Wild Mule (Jimenez, Albin)    118 L  4.0    3.0   2   2   2   
1      Rol Again Dancer (Rodriguez, Angel)    118 L  5.0    2.0   3   3   1   
2           Bright Forecast (Ferrer, Jose)    118 L  1.0    5.0   5   5   4   
3  Running Right by U (Vargas, Jr., Jorge)    118 L  2.0    4.0   4   4   5   
4          Run Happy Pappy (Torres, Jomar)  118 L b  3.0    1.0   1   1   3   

  Fin   Odds               Comments  
0   1  0.90*  battled back, got nod  
1   2   1.50   vied 3w, led, missed  
2   3   4.70    off slowly, rallied  
3   4  14.40       inside, no rally  
4   5  14.10   dueled inside, faded  
   index   Last Raced  Unnamed: 0  Unnamed: 

In [15]:
cleaned_df_list[5]

,Pgm,Horse Name,Start,3/16,3/8,Str,Fin,3/16_length_behind,3/8_length_behind,Str_length_behind,Fin_length_behind
0,2.0,Recruiter,1.0,1,1,1,1,2 1/2,5.0,3 1/2,1
1,7.0,Global Surprise,3.0,3,2,2,2,NaN,NaN,NaN,NaN
2,8.0,Cruising Derek,2.0,2,3,3,3,6 1/2,5.0,3 1/2,1
3,6.0,Capt Complicated,5.0,5,4,4,4,NaN,NaN,NaN,NaN
4,1.0,Give God Praise,4.0,4,5,5,5,2 1/2,6.0,9 1/2,16 1/4
